You now know the following

1. Generate open-loop control from a given route

2. Simulate vehicular robot motion using bicycle/ unicycle model

Imagine you want to make an utility for your co-workers to try and understand vehicle models. 
Dashboards are common way to do this.

There are several options out there : Streamlit, Voila, Observable etc

Follow this
<a href="https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e">Medium post</a> on Jupyter Dash and see how to package what you learnt today in an interactive manner

Here is a   <a href="https://stackoverflow.com/questions/53622518/launch-a-dash-app-in-a-google-colab-notebook">stackoverflow question </a> on how to run dash applications on Collab

What can you assume?
+ Fix $v,\omega$ or $v,\delta$ depending on the model (users can still pick the actual value)
+ fixed wheelbase for bicycle model

Users can choose 
+ unicycle and bicycle models
+ A pre-configured route ("S", "inverted-S", "figure-of-eight" etc)
+ 1 of 3 values for $v, \omega$ (or $\delta$) 

In [1]:
!pip install jupyter-dash

     |████████████████████████████████| 81kB 2.7MB/s 
     |████████████████████████████████| 1.0MB 16.3MB/s 
     |████████████████████████████████| 3.5MB 33.9MB/s 
     |████████████████████████████████| 92kB 10.0MB/s 
     |████████████████████████████████| 1.8MB 29.6MB/s 
     |████████████████████████████████| 358kB 35.4MB/s 
  Created wheel for dash: filename=dash-1.20.0-cp37-none-any.whl size=85845 sha256=a612111c426e04c777fa74da1fda59c9a1f9b72db25e55894205d6674500edc2
  Stored in directory: /root/.cache/pip/wheels/e6/33/41/ce361d0d1da277d2350f815b10f2ab1e331592d93ef2b07e29
  Created wheel for dash-renderer: filename=dash_renderer-1.9.1-cp37-none-any.whl size=1014873 sha256=78338f5112b46a91741f48035a1f7af422566570a9656d6eee0d818a1ccfdc9f
  Stored in directory: /root/.cache/pip/wheels/03/a9/c5/dd5815c601b0ede164c223ffd7bafebde716ca57de06ef8aec
  Created wheel for dash-core-components: filename=dash_core_components-1.16.0-cp37-none-any.whl size=3540992 sha256=ddbd059b293bd0308de45

In [2]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np

# Load Data
velocities = ['1','2','3']
omegas = ['15','30','45']
shapes = ["S", "Inverted-S", "Figure of 8"]
models = ["Unicycle", "Bicycle"]

def unicycle_model(curr_pose, v, w, dt=1.0):
    '''
    >>> unicycle_model((0.0,0.0,0.0), 1.0, 0.0)
    (1.0, 0.0, 0.0)
    >>> unicycle_model((0.0,0.0,0.0), 0.0, 1.0)
    (0.0, 0.0, 1.0)
    >>> unicycle_model((0.0, 0.0, 0.0), 1.0, 1.0)
    (1.0, 0.0, 1.0)
    '''
    ## write code to calculate next_pose
    # refer to the kinematic equations of a unicycle model
    x, y, theta = curr_pose
    x += v*np.cos(theta)*dt
    y += v*np.sin(theta)*dt
    theta += w*dt
    
    # Keep theta bounded between [-pi, pi]
    theta = np.arctan2(np.sin(theta), np.cos(theta))

    # return calculated (x, y, theta)
    return x, y, theta
  
def bicycle_model(curr_pose, v, delta, dt=1.0):
    '''
    >>> bicycle_model((0.0,0.0,0.0), 1.0, 0.0)
    (1.0, 0.0, 0.0)
    >>> bicycle_model((0.0,0.0,0.0), 0.0, np.pi/4)
    (0.0, 0.0, 0.0) 
    >>> bicycle_model((0.0, 0.0, 0.0), 1.0, np.pi/4)
    (1.0, 0.0, 1.11) 
    '''
    # write code to calculate next_pose
    # refer to the kinematic equations of a bicycle model
    #x, y, theta = 
    #x =
    #y =
    #theta =
    L = 0.9
    x, y, theta = curr_pose 
    x += v*np.cos(theta)*dt 
    y += v*np.sin(theta)*dt
    theta += (v/L)*np.tan(delta)*dt
    # Keep theta bounded between [-pi, pi]
    theta = np.arctan2(np.sin(theta), np.cos(theta))
    # return calculated (x, y, theta)
    return x, y, theta

def get_open_loop_commands(route, vc_fast=1, wc=np.pi/12, dt=1.0):
    all_w = []
    omegas = {'straight': 0, 'left': wc, 'right': -wc}
    for manoeuvre, command in route:
      u = np.ceil(command/vc_fast).astype('int')
      v = np.ceil(np.deg2rad(command)/wc).astype('int')
      t_cmd = u if manoeuvre == 'straight' else v
      all_w += [omegas[manoeuvre]]*t_cmd
    all_v = vc_fast * np.ones_like(all_w)
    return all_v, all_w

def get_commands(shape):
    if(shape == shapes[0]):
        return [("right", 180),("left", 180)]
    elif(shape == shapes[1]):
        return [("left", 180),("right", 180)]
    return [("right", 180),("left", 180),("left", 180),("right", 180)]

def get_angle(omega):
  if(omega == omegas[0]):
    return np.pi/12
  elif(omega == omegas[1]):
    return np.pi/6
  return np.pi/4


# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Unicycle/Bicycle"),
    html.Label([
        "velocity",
        dcc.Dropdown(
            id='velocity', clearable=False,
            value='1', options=[
                {'label': c, 'value': c}
                for c in velocities
            ])
    ]),
    html.Label([
        "omega/delta",
        dcc.Dropdown(
            id='omega', clearable=False,
            value='15', options=[
                {'label': c, 'value': c}
                for c in omegas
            ])
    ]),
    html.Label([
        "shape",
        dcc.Dropdown(
            id='shape', clearable=False,
            value='S', options=[
                {'label': c, 'value': c}
                for c in shapes
            ])
    ]),
    html.Label([
        "model",
        dcc.Dropdown(
            id='model', clearable=False,
            value='Unicycle', options=[
                {'label': c, 'value': c}
                for c in models
            ])
    ]),
    dcc.Graph(id='graph'),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("velocity", "value"), Input("omega", "value"), Input("shape", "value"), Input("model", "value")]
)
def update_figure(velocity, omega, shape, model):
    robot_trajectory = []
    all_v, all_w = get_open_loop_commands(get_commands(shape), int(velocity), get_angle(omega))
    pose = (0, 0, np.pi/2)
    for v, w in zip(all_v, all_w):
        robot_trajectory.append(pose)
        if model == models[0]:
          pose = unicycle_model(pose, v, w)
        else:
          pose = bicycle_model(pose,v,w)
    robot_trajectory = np.array(robot_trajectory)
    dt = pd.DataFrame({'x-axis': robot_trajectory[:,0],'y-axis': robot_trajectory[:,1]})
    return px.line(dt, x="x-axis", y="y-axis", title='Simulate vehicular robot motion using unicycle/bicycle model')
# Run app and display result inline in the notebook
app.run_server(mode='inline')

<IPython.core.display.Javascript object>